In [77]:
import pandas as pd

# Load data
df = pd.read_csv('votes.csv')

# Convert the 'Created' column to datetime
df['Created'] = pd.to_datetime(df['Created'])

# Set the 'Created' column as the DataFrame's index
df.set_index('Created', inplace=True)

# Filter the DataFrame for entries from 2022
df = df[df.index.year == 2022]

# Reset the index
df.reset_index(inplace=True)

# Get a list of snapshot IDs for each member
snapshot_list = df.groupby('Member')['Snapshot ID'].apply(lambda x: tuple(sorted(x))).reset_index()

# Create a unique identifier for each list of snapshot IDs
snapshot_list['Snapshot_IDs'] = snapshot_list['Snapshot ID'].astype('str')

# Group members who have the exact same snapshot IDs
member_clusters_exact = snapshot_list.groupby('Snapshot_IDs')['Member'].apply(list).reset_index()

# Filter out groups with only one member and one Snapshot ID
member_clusters_exact = member_clusters_exact[member_clusters_exact['Member'].apply(len) > 1]
member_clusters_exact = member_clusters_exact[member_clusters_exact['Snapshot_IDs'].apply(lambda x: len(eval(x)) > 1)]

# Assign a unique cluster ID to each group
member_clusters_exact['Cluster_ID'] = range(1, len(member_clusters_exact) + 1)

# Calculate number of members in each cluster
member_clusters_exact['Number_of_Members'] = member_clusters_exact['Member'].apply(len)

# Rearrange columns
member_clusters_exact = member_clusters_exact[['Member', 'Number_of_Members', 'Cluster_ID', 'Snapshot_IDs']]

# Rename columns
member_clusters_exact.columns = ['Members', 'Number_of_Members', 'Cluster_ID', 'Snapshot_IDs']

print(member_clusters_exact)
member_clusters_exact.to_csv('member_clusters_exact.csv', index=False)


                                                Members  Number_of_Members  \
141   [0x1904998e186635ed3dFf18D3Ce867CDD7C0B8A97, 0...                  6   
249   [0x89E597b6123c25eFD2ce6c1084eAdd8d624b7D9A, 0...                  2   
257   [0x3d17c3f656d1EFbe7b21FAbb13535B0863b220B2, 0...                  2   
444   [0x0147e150D6bcBB2150959ba4a8b16f7a9D3fEe84, 0...                  3   
482   [0x5f9aAaCd65Ab18DcCAb83703B3cbf0a983D91132, 0...                  2   
486   [0x1278B2888B1C8990cA843991D7062a4f6126e969, 0...                  3   
488   [0xa24f508328cD22aBccD3E07739F2C32e99a25d33, 0...                  4   
492   [0x063038777Bb5BE889f092eBaB343F423Aef9691d, 0...                 35   
493   [0x43b4FEC74deaB75a6C77da2BbA1Bc17bb7BaDb03, 0...                  2   
547   [0x81AC664E57C682C64414013Fc9Cb58088D3404EC, 0...                  2   
593   [0x56decf93dA542a4B1fC17f6F4d65d4B829Ce0B35, 0...                  6   
619   [0x1CA5E86A1a7a88645DFE163071367A317C9033e4, 0...         

In [85]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, fcluster

# Load data
df = pd.read_csv('votes.csv')

# Convert the 'Created' column to datetime
df['Created'] = pd.to_datetime(df['Created'])

# Set the 'Created' column as the DataFrame's index
df.set_index('Created', inplace=True)

# Filter the DataFrame for entries from 2022
df = df[df.index.year == 2022]

# Reset the index
df.reset_index(inplace=True)

# Create binary matrix
members = df['Member'].unique()
snapshots = df['Snapshot ID'].unique()
matrix = pd.DataFrame(0, index=members, columns=snapshots)
for _, row in df.iterrows():
    matrix.loc[row['Member'], row['Snapshot ID']] = 1

# Perform hierarchical clustering
distances = pdist(matrix.values, metric='jaccard')
clusters = fcluster(linkage(distances, method='complete'), 0.5, criterion='distance')

# Assign cluster labels to members
member_clusters = pd.DataFrame({'Member': members, 'Cluster_ID': clusters})

# Get a list of snapshot IDs for each member
snapshot_list = df.groupby('Member')['Snapshot ID'].apply(lambda x: tuple(sorted(x))).reset_index()

# Merge snapshot list with member clusters
member_clusters = member_clusters.merge(snapshot_list, on='Member')

# Group by 'Cluster_ID' and combine 'Snapshot ID' into a list
member_clusters = member_clusters.groupby('Cluster_ID').agg({'Member': lambda x: list(x), 'Snapshot ID': lambda x: list(x)}).reset_index()

# Add a column showing the number of members in each cluster
member_clusters['Number_of_Members'] = member_clusters['Member'].apply(len)

# Filter out groups with only one member
member_clusters = member_clusters[member_clusters['Number_of_Members'] > 1]

# Filter out groups with only one unique Snapshot ID
member_clusters['Number_of_Snapshot_IDs'] = member_clusters['Snapshot ID'].apply(lambda x: len(set(x)))
member_clusters = member_clusters[member_clusters['Number_of_Snapshot_IDs'] > 1]

# Re-assign cluster ID starting from 1
member_clusters['Cluster_ID'] = range(1, len(member_clusters) + 1)

# Rearrange columns
member_clusters = member_clusters[['Member', 'Number_of_Members', 'Cluster_ID', 'Snapshot ID']]

# Rename columns
member_clusters.columns = ['Members', 'Number_of_Members', 'Cluster_ID', 'Snapshot_IDs']

print(member_clusters)
member_clusters.to_csv('member_clusters_hierarchical.csv', index=False)


                                                Members  Number_of_Members  \
2     [0xDFa951ebCBea44Be7dB7FB2848F086365FB55483, 0...                  2   
3     [0xDCe7D7F3eA933B214b1E73B47B079b631122596e, 0...                  3   
5     [0xf5539cc0E7df19a77299d740f8D7A845863df99C, 0...                  2   
10    [0x3aa39669Df49eC091F27B47a9EB4Ef00939DCc5d, 0...                  3   
15    [0x6C027d357C7C8c520Bc328532C99aaAAe341B6B8, 0...                  2   
...                                                 ...                ...   
1537  [0xDD8E8B66613CB347CF2460f692894554836f76E9, 0...                  4   
1547  [0x3FBdAD0511aB95BEEBE56c8193791D20af66027d, 0...                  3   
1553  [0xa9817CA5dEBC329d94568e0054DccE75Fae6386d, 0...                  2   
1556  [0xd2172778f6132B2619DC07d71A0a0F068e0883C3, 0...                  2   
1559  [0x5AcEb679be75B33df9216B2a25dc88Ceb26cD305, 0...                  2   

      Cluster_ID                                       Snapshot

In [86]:
def interpret_choice(choice):
    choice_lower = str(choice).lower()
    if 'yes' in choice_lower:
        return 'Support'
    elif 'no' in choice_lower:
        return 'Oppose'
    else:
        return 'Neutral'

# Apply function to 'Choice' column
df['Support/Oppose'] = df['Choice'].apply(interpret_choice)

# Filter dataframe to only include rows where the choice contains 'yes' or 'no'
df = df[df['Choice'].str.contains('yes', case=False) | df['Choice'].str.contains('no', case=False)]

# Count total votes per proposal
proposal_votes = df.groupby('Proposal Title').size().reset_index(name='Total Votes')

# Calculate the count of each sentiment
sentiment_counts = df.groupby(['Proposal Title', 'Choice', 'Support/Oppose']).size().reset_index(name='Counts')

# Calculate the percentage of each sentiment
sentiment_counts['Percentage'] = sentiment_counts.groupby('Proposal Title')['Counts'].apply(lambda x: 100 * x / x.sum())

# Merge total votes into sentiment_percentages
result = pd.merge(sentiment_counts, proposal_votes, on='Proposal Title')

# Save the result as a csv file
result.to_csv('proposal_votes_analysis.csv', index=False)

/var/folders/6k/dv9js2xs2gd4h3jcz0l35g6c0000gn/T/ipykernel_6932/2945635035.py:23: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sentiment_counts['Percentage'] = sentiment_counts.groupby('Proposal Title')['Counts'].apply(lambda x: 100 * x / x.sum())


In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('votes.csv')

# Drop unnecessary columns
df.drop(columns=['Vote Weight', 'Total VP', 'MANA VP', 'Names VP', 'LAND VP', 'Delegated VP', 'L1 Wearables VP', 'Rental VP'], inplace=True)

# Convert the 'Created' column to datetime
df['Created'] = pd.to_datetime(df['Created'])

# Set the 'Created' column as the DataFrame's index
df.set_index('Created', inplace=True)

# Filter the DataFrame for entries from 2022
df = df[df.index.year == 2022]

# Reset the index
df.reset_index(inplace=True)

# Get a list of (snapshot ID, created time) tuples for each member
df['Snapshot_Created'] = list(zip(df['Snapshot ID'], df['Created']))
snapshot_list = df.groupby('Member')['Snapshot_Created'].apply(list).reset_index()

# Create a unique identifier for each list of snapshot IDs
snapshot_list['Snapshot_IDs'] = snapshot_list['Snapshot_Created'].apply(lambda x: set([y[0] for y in x])).astype('str')

# Group members who have the exact same set of snapshot IDs
member_clusters_exact = snapshot_list.groupby('Snapshot_IDs').agg({'Member': list}).reset_index()

# Filter out groups with only one member and one Snapshot ID
member_clusters_exact['Number_of_Members'] = member_clusters_exact['Member'].apply(len)
member_clusters_exact = member_clusters_exact[member_clusters_exact['Number_of_Members'] > 1]
member_clusters_exact['Number_of_Snapshots'] = member_clusters_exact['Snapshot_IDs'].apply(lambda x: len(eval(x)))
member_clusters_exact = member_clusters_exact[member_clusters_exact['Number_of_Snapshots'] > 1]

# Assign a unique cluster ID to each group
member_clusters_exact['Cluster_ID'] = range(1, len(member_clusters_exact) + 1)

# Expand the "Members" list into separate rows
expanded = member_clusters_exact.apply(lambda x: pd.Series(x['Member']), axis=1).stack().reset_index(level=1, drop=True)
expanded.name = 'Member'
member_clusters_expanded = member_clusters_exact.drop('Member', axis=1).join(expanded)

# Merge back with original DataFrame to get vote times for each member and snapshot
expanded_votes = df.merge(member_clusters_expanded[['Member', 'Cluster_ID', 'Snapshot_IDs']], left_on='Member', right_on='Member')

# Drop the Snapshot_Created column and keep only the Created (time) column
expanded_votes.drop(columns=['Snapshot_Created'], inplace=True)

# Sort by "Created" within each "Cluster_ID" and "Snapshot ID" and assign "Vote Order"
expanded_votes.sort_values(by=['Cluster_ID', 'Snapshot ID', 'Created'], inplace=True)
expanded_votes['Vote Order'] = expanded_votes.groupby(['Cluster_ID', 'Snapshot ID']).cumcount() + 1

# Determine the leader of each snapshot within each cluster (the member who voted first)
leaders = expanded_votes[expanded_votes['Vote Order'] == 1][['Cluster_ID', 'Snapshot ID', 'Member']]
leaders.columns = ['Cluster_ID', 'Snapshot ID', 'Leader']

expanded_votes = expanded_votes.merge(leaders, on=['Cluster_ID', 'Snapshot ID'])

# Reorder columns
columns_order = ['Snapshot_IDs', 'Member', 'Cluster_ID', 'Snapshot ID', 'Created', 'Vote Order', 'Leader']
expanded_votes = expanded_votes[columns_order]

print(expanded_votes)
expanded_votes.to_csv('opinion_leader.csv', index=False)
